## Codificación de las recompensas

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Recompensas de codificación

- Ya hemos hablado de la importancia de codificar las observaciones.
- También podemos tener alguna opción en el espacio de acción, aunque aquí (y a menudo) es relativamente clara/fija.
- Pero, ¿qué pasa con las recompensas? 

#### Configuración actual

- Actualmente, obtenemos una recompensa de +1 por alcanzar el objetivo 
- Esto es parte de lo que hace que RL sea tan difícil (e impresionante):
  - Queremos aprender sobre las acciones aunque no sepamos de inmediato si la acción fue "buena" 
  - Contrasta esto con el aprendizaje supervisado, donde cada predicción que hacemos sobre los datos de entrenamiento puede compararse inmediatamente con el valor objetivo conocido.


In [2]:
# TODO: perhaps this next slide can be moved to Module 1, since it's very general?

#### Los agentes no pueden ser simplemente codiciosos

- ¿Pueden los agentes aprender simplemente a buscar la mejor recompensa inmediata?
- No. Por ejemplo, en un sistema de recomendación de vídeos, mostrar al usuario otro vídeo divertido de gatos puede hacer que haga clic (alta recompensa inmediata) pero provocar una pérdida de interés en el servicio a largo plazo (baja recompensa a largo plazo).
- Nuestro Lago Helado es otro ejemplo del problema: a veces no hay ninguna recompensa inmediata de la que aprender.

In [3]:
# TODO: perhaps this next section on "Learned action probabilities" could be moved much earlier, even as early as Module 1

#### Probabilidades de acción aprendidas

- RLlib nos permite mirar dentro del modelo la probabilidad de cada acción dada una observación (es decir, la política aprendida).
- Carguemos el modelo entrenado con nuestras observaciones codificadas:

In [4]:
from envs_03 import RandomLakeObs
from ray.rllib.algorithms.ppo import PPOConfig

ppo_config = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")
)
ppo_RandomLakeObs = ppo_config.build(env=RandomLakeObs)

In [5]:
# # HIDDEN

# for i in range(16):
#     ppo_RandomLakeObs.train()
    
# print(ppo_RandomLakeObs.evaluate()["evaluation"]["episode_reward_mean"])

# ppo_RandomLakeObs.save("models/RandomLakeObs-Ray2")

In [6]:
ppo_RandomLakeObs.restore("models/RandomLakeObs-Ray2/checkpoint_000016")

#### Probabilidades de acción aprendidas

Utilizaremos la función `query_Policy` del módulo 2:

In [7]:
from utils_03 import query_policy
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,0,0,0])

array([0.00902206, 0.5078786 , 0.47434822, 0.00875122], dtype=float32)

- Recuerda la ordenación (izquierda, abajo, derecha, arriba).
- Cuando la observación es `[0 0 0]` (sin agujeros ni aristas a la vista), el agente prefiere ir hacia abajo y hacia la derecha.

¿Y si hay un agujero debajo de ti? Podemos introducir una observación diferente en la política:

In [8]:
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,1,0,0])

array([0.01965651, 0.00299437, 0.9645823 , 0.01276694], dtype=float32)

- ¡Ahora es muy poco probable que el agente baje, y muy probable que vaya a la derecha!
- De nuevo, todo esto se aprendió por ensayo y error, con una recompensa obtenida sólo cuando se alcanzaba el objetivo.

#### Recompensas de Random Lake

- En el ejemplo de Random Lake, ¿no se puede facilitar la vida al agente dando recompensas inmediatas?

Este es el código de recompensa actual:

In [9]:
def reward(self):
    return int(self.player == self.goal)

- El agente tiene que aprender, por ensayo y error a lo largo de _episodios enteros_, que moverse hacia abajo y hacia la derecha es generalmente algo bueno 

#### Redefinir las recompensas

- Intentemos, en cambio, dar una recompensa _en cada paso, que sea mayor a medida que el agente se acerca al objetivo_ 

In [10]:
from envs_03 import RandomLakeObs

class RandomLakeObsRew(RandomLakeObs):
    def reward(self):
        return 6-(abs(self.player[0]-self.goal[0]) + abs(self.player[1]-self.goal[1]))

- El método anterior utiliza la [Distancia Manhattan](https://en.wikipedia.org/wiki/Taxicab_geometry) entre el jugador y la meta como recompensa 
- Cuando el agente llega a la meta, se obtiene la recompensa máxima de 6.
- Cuando el agente se aleja más de la meta, se obtiene la recompensa mínima de 0.

#### Redefinir las recompensas

In [11]:
env = RandomLakeObsRew()
env.reset()
env.render()
env.reward()

🧑🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊⛳️


0

⬆️ la recompensa es 0

⬇️ la recompensa es 1 porque nos acercamos al objetivo

In [12]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧑🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊⛳️


1

#### Redefinir las recompensas

In [13]:
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧑
🧊🧊🧊⛳️


5

Ahora, la recompensa es 5. Después, será 6.

In [14]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊🧑


6

#### Comparar las recompensas

- Así pues, tenemos dos posibles funciones de recompensa. ¿Cuál funciona mejor? 
- Recuerda que la última vez, después de entrenar durante 8 iteraciones, fuimos capaces de alcanzar el objetivo alrededor del 70% de las veces:

In [15]:
ppo_RandomLakeObs.evaluate()['evaluation']['episode_reward_mean']

0.8166666666666667

#### Comparando las recompensas

¡Vamos a entrenar con la nueva función de recompensa!

In [16]:
ppo_RandomLakeObsRew = ppo_config.build(env=RandomLakeObsRew)

In [17]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [18]:
ppo_RandomLakeObsRew.evaluate()['evaluation']['episode_reward_mean']

101.90566037735849

Un momento, ¿qué está pasando aquí?

#### ¿Comparar las recompensas?

- Intentamos mejorar nuestro sistema de RL dando forma a la función de recompensa.
- Esto (presumiblemente) afectó al entrenamiento, pero también a nuestra evaluación.
- En el aprendizaje supervisado, esto es como cambiar la métrica de puntuación del error cuadrado al error absoluto.
- Si el antiguo sistema obtuvo un error medio al cuadrado de 20.000 y el nuevo sistema obtuvo un error medio absoluto de 40, ¿cuál es mejor?
- ¡Estamos comparando manzanas y naranjas!
- Queremos comparar ambos modelos con la misma métrica, por ejemplo la métrica original 
- En este caso, queremos ver la frecuencia con la que el agente alcanza el objetivo.

#### ¿Comparar las recompensas?

- El código aquí es un poco más avanzado.
- Se incluye para completarlo, pero no entraremos en detalles.

In [19]:
from ray.rllib.agents.callbacks import DefaultCallbacks

class MyCallbacks(DefaultCallbacks):
    def on_episode_end(self, *, worker, base_env, policies, episode, env_index, **kwargs):
        info = episode.last_info_for()
        episode.custom_metrics["goal_reached"] = info["player"] == info["goal"]

In [20]:
ppo_config_callback = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")\
    .callbacks(callbacks_class=MyCallbacks)\
    .evaluation(evaluation_config={"callbacks" : MyCallbacks})
)

ppo_RandomLakeObsRew = ppo_config_callback.build(env=RandomLakeObsRew)

El entrenador de arriba utiliza nuestro nuevo esquema de recompensas, pero también informa/mede la tasa de consecución del objetivo.

#### ¿Comparar las recompensas?

¡Vamos a probarlo!

In [21]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [22]:
# HIDDEN
ppo_RandomLakeObsRew.evaluate()["evaluation"]["episode_reward_mean"]

101.90566037735849

In [23]:
ppo_RandomLakeObsRew.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.04081632653061224

- Hmm, ¡estos resultados son terribles!
- Antes obteníamos un porcentaje de victorias superior al 70%, y ahora estamos cerca de cero.
- ¿Qué ha pasado? 🤔

#### ¿Qué está optimizando realmente el agente?

- El agente está optimizando realmente la _recompensa total descontada_.
- _Total_: valora todas las recompensas que recoge, no sólo la recompensa final.
- _Descontado_: valora más las recompensas anteriores que las posteriores.
- Nuestro agente está maximizando con éxito la recompensa total descontada, pero esto no se corresponde con alcanzar el objetivo.
- ¿Pero por qué? La meta da una recompensa mayor.

#### Exploración frente a explotación

- Un concepto fundamental en la RL es _exploración vs. explotación_
- Cuando el agente está aprendiendo la política, puede elegir entre

1. Hacer cosas que sabe que son bastante buenas ("explotar")
2. Probar algo totalmente nuevo y loco, por si acaso ("explorar")

In [24]:
# TODO 
# diagram for this?

#### Exploración frente a explotación

- Con la antigua estructura de recompensa, el agente obtiene una recompensa de 0 a menos que alcance el objetivo.
  - Por tanto, sigue intentando encontrar algo mejor.
- Con la nueva estructura de recompensa, el agente obtiene mucha recompensa sólo por caminar.
  - No está muy motivado para explorar el entorno.
- De hecho, como está maximizando la recompensa **total** descontada, ¡encontrar la meta es algo malo!
  - Esto hace que el episodio termine, limitando la recompensa total del agente.
  - En realidad, el agente aprende a _evitar_ la meta, especialmente al principio del episodio.

#### Diseñar una mejor estructura de recompensas

- En su lugar, intentemos penalizar al agente cuando se meta en un agujero o se salga del borde.
- Será más fácil ponerlo en práctica directamente en el "paso":

In [25]:
class RandomLakeObsRew2(RandomLakeObs):
    def step(self, action):
        # (not shown) existing code gets new_loc, where the player is trying to go
        
        reward = 0
        
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        else:
            reward -= 0.1 # small penalty
            
        if self.holes[self.player]:
            reward -= 0.1 # small penalty
            
        if self.player == self.goal:
            reward += 1
        
        # Return observation/reward/done
        return self.observation(), reward, self.done(), {"player" : self.player, "goal" : self.goal}

In [26]:
# HIDDEN
from envs_03 import RandomLakeObsRew2

#### Probando, de nuevo

In [27]:
# HIDDEN
# redefine ppo_RandomLakeObs to include the new callbacks
# so that you can measure the custom metric instead of the reward
# they will give the same value but this is better for consistency
ppo_RandomLakeObs = ppo_config_callback.build(env=RandomLakeObs)

for i in range(8):
    ppo_RandomLakeObs.train()

In [30]:
ppo_RandomLakeObsRew2 = ppo_config_callback.build(env=RandomLakeObsRew2)

In [31]:
for i in range(8):
    ppo_RandomLakeObsRew2.train()

In [32]:
ppo_RandomLakeObs.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.6853448275862069

In [33]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.734982332155477

Parece que, esta vez, los dos métodos tienen un rendimiento mucho más similar.

#### Duración del episodio

- Además de la tasa de éxito, podemos calcular otras estadísticas del comportamiento del agente.
- Una medida interesante es la duración del episodio.
- RLlib la registra por defecto, por lo que podemos acceder a ella fácilmente:

In [34]:
ppo_RandomLakeObs.evaluate()["evaluation"]["episode_len_mean"]

8.585470085470085

In [35]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["episode_len_mean"]

7.20216606498195

Aunque los dos agentes tienen el mismo porcentaje de éxito, el nuevo tiende a tener episodios más cortos.

Notas 

- Esto es bastante interesante porque el agente no puede "ver" la diferencia entre los agujeros y las aristas.
- Podríamos explorar esto más a fondo añadiendo más métricas personalizadas, por ejemplo, el número de baches en la arista.

In [ ]:
# TODO
#### disadvantages - loss of generality

#- now only works if goal is at bottom-right
#give a few real-world examples here -> important

## Analogía del aprendizaje supervisado: la conformación de la recompensa
<!-- multiple choice -->

Antes hemos hecho una analogía entre la codificación de observaciones en la LR y el preprocesamiento de características en el aprendizaje supervisado. ¿Qué aspecto del aprendizaje supervisado es la mejor analogía con la conformación de la recompensa en la RL?

- [ ] Ingeniería de rasgos 
- [ ] Selección de modelos | No exactamente. Pero, como veremos, ¡la selección de modelos también tiene cabida en la LR!
- [Ajuste de los hiperparámetros: No del todo. Pero, como veremos, ¡el ajuste de hiperparámetros también tiene cabida en la RL!
- [x] Selección de una función de pérdida | Cambiar la función de pérdida cambia el "mejor" modelo, igual que cambiar las recompensas cambia la "mejor" política.

## Premiar cada paso: pequeñas recompensas negativas
<!-- multiple choice -->

En entornos de RL como Random Lake, en los que el agente debe alcanzar un objetivo concreto, imagina que asignamos una pequeña recompensa negativa por _cada_ paso que da el agente. ¿Cómo afectaría esto, en general/típicamente, a la cantidad de tiempo que el agente emplea hasta alcanzar el objetivo?

- [x] El agente intentará llegar a la meta en el menor número de pasos posible.
- [ ] El agente intentará llegar a la meta en el mayor número de pasos posible. | Si estamos penalizando cada paso, dar más pasos supondrá una menor recompensa.
- [ ] No hay cambios. | Si estamos penalizando cada paso, dar más pasos dará lugar a una menor recompensa.

## Exploración vs. Explotación
<!-- multiple choice -->

¿Cuál de las siguientes es una afirmación correcta sobre el compromiso de exploración-explotación en la LR?

- [ ] Si un sólo explora, nunca encontrará una buena política. | De hecho, encontrará buenas políticas, sólo que de forma EXTREMADAMENTE lenta.
- [x] Si un agente sólo explota, nunca encontrará una buena política. | Puede que siga intentando lo mismo una y otra vez.
- [ ] Los agentes siempre encuentran buenas políticas incluso sin exploración/explotación.

## Consecuencias imprevistas
<!-- coding exercise -->

En este ejercicio, probarás una mala idea: asignar una gran recompensa negativa cada vez que el agente dé un paso. Utilizaremos -1 por paso. El agente sigue recibiendo una recompensa de +1 por alcanzar la meta. Implementa esta recompensa, entrena al agente y observa la duración media de los episodios que imprime el código. Compáralo con la duración media de los episodios de un agente que sólo actúa al azar. Luego, responde a la pregunta de opción múltiple sobre el comportamiento del agente. ¿Qué crees que ocurre aquí? 

(Para tu información: como se ha comentado anteriormente, este tipo de cambio en un entorno también se puede conseguir con envoltorios de gimnasia)

In [ ]:
# EXERCISE
from utils_03 import lake_default_config
from envs_03 import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return ____
    
ppo = lake_default_config.build(env=____)

for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"][____])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"][____])

In [2]:
# SOLUTION
from utils_03 import lake_default_config
from envs_03 import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return old_reward - 1

ppo = lake_default_config.build(env=RandomLakeBadIdea)


for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"]["episode_len_mean"])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"]["episode_len_mean"])

0
1
2
3
4
5
6
7
Average episode length for trained agent: 4.4
Average episode length for random agent: 12.1


#### Comportamiento del agente

Cuando se entrena en un entorno con una gran recompensa negativa a cada paso, ¿qué crees que hace este agente, que es indeseable?

- [ ] El agente se queda quieto porque se le disuade de moverse. | ¡Inténtalo de nuevo!
- [ ] El agente no está interesado en alcanzar el objetivo porque la recompensa es comparativamente pequeña. | ¡Inténtalo de nuevo!
- [x] El agente aprende a saltar al lago tan rápido como puede, para evitar la recompensa negativa de moverse. | ¡Caramba! 🥶
- [ ] El agente llega a la meta enseguida. | ¡Sin embargo, sería deseable!